In [ ]:
!pip install miceforest

In [2]:
#Library Imports

import pandas as pd
import numpy as np
import miceforest as mf

In [121]:
#Functions

#Read and combine datasets
def read_data(path1,path2):
    """
    Parameters:
    path1 (str): file path of first csv file.
    path2 (str): file path of second csv file.

    Returns:
    pandas.DataFrame: combined dataframe of both csv files.
    """
    df1 = pd.read_csv(path1) #crime data from 2010 to 2019
    df2 = pd.read_csv(path2) # crime data from 2022
    df1.rename(columns = {'AREA ':'AREA'},inplace=True)
    df = pd.concat([df1,df2])
    return df

#Dropping columns with more than 60% NaN values
def drop_columns(df,dropped_columns):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.
    dropped_columns (list): list of column names to be dropped.

    Returns:
    pandas.DataFrame: dataframe with specified columns removed.
    """

    df = df.drop(dropped_columns,axis=1)
    return df

#Preprocessing the datatime columns
def preprocess_datetime(df,column_name):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.
    column_name (str): name of column to be converted to datetime.

    Returns:
    pandas.DataFrame: dataframe with specified column converted to datetime format.
    """

    df[column_name] = pd.to_datetime(df[column_name],infer_datetime_format=True)
    return df

#Padding the time occured column
def pad_time(df):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.

    Returns:
    pandas.DataFrame: dataframe with 'TIME OCC' column padded with leading zeros.
    """

    df['TIME OCC'] = df['TIME OCC'].astype(str).str.zfill(4)
    return df

#Data Imputation
def impute_data(df):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.

    Returns:
    pandas.DataFrame: dataframe with imputed data.
    """

    df_impute = pd.DataFrame(df[['AREA','Rpt Dist No','Crm Cd','Vict Age','Vict Sex','Vict Descent','Premis Cd','Status']])
    df_impute['Vict Sex'] = df_impute['Vict Sex'].astype('category')
    df_impute['Vict Descent'] = df_impute['Vict Descent'].astype('category')
    df_impute['Status'] = df_impute['Status'].astype('category')
    kernel = mf.ImputationKernel(
    data=df_impute,
    save_all_iterations=True,
    random_state=1991
    )
    kernel.mice(3,verbose=True)
    complete_df = kernel.complete_data()
    df['Status'] = complete_df['Status']
    df['Premis Cd'] = complete_df['Premis Cd']
    df['Vict Sex'] = complete_df['Vict Sex']
    df['Vict Descent'] = complete_df['Vict Descent']
    return df
    
    
#Create lookup dictionary
def create_lookup_dict(df, data, code):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.
    data (str): column name containing data values.
    code (str): column name containing code values.

    Returns:
    dict: dictionary with data values as keys and corresponding code values as values.
    """

    lookup_dict = {}
    for idx, row in df.iterrows():
        if pd.isnull(row[data]):
            if row[code] in lookup_dict:
                lookup_dict[row[code]] = lookup_dict[row[code]]
            else:
                lookup_dict[row[code]] = row[code]
        else:
            lookup_dict[row[data]] = row[code]
    return lookup_dict

#Outlier treatement
def remove_outliers(df):
    """
    Parameters:
    df (pandas.DataFrame): dataframe containing data.

    Returns:
    pandas.DataFrame: dataframe with outliers removed.
    """

    df = df[df['Vict Age'] >= 0]
    df = df[(df['LAT'] != 0) & (df['LON'] != 0)]
    df = df[df['Vict Descent'] != '-']
    df = df[(df['Vict Sex'] != '-') & (df['Vict Sex'] != 'H') & (df['Vict Sex'] != 'N')]
    return df

In [4]:
path1 = 'Crime_Data_from_2010_to_2019.csv'
path2 = 'Crime_Data_from_2020_to_Present.csv'
df = read_data(path1,path2)

In [5]:
dropped_columns = ['Crm Cd 2','Crm Cd 3','Crm Cd 4','Cross Street','Mocodes','Weapon Desc','Weapon Used Cd']
df = drop_columns(df,dropped_columns)

In [115]:
df = preprocess_datetime(df,"Date Rptd")
df = preprocess_datetime(df,"DATE OCC")
df = pad_time(df)

In [92]:
crime_code_dict = create_lookup_dict(df, 'Crm Cd Desc', 'Crm Cd')
premise_code_dict = create_lookup_dict(df,'Premis Desc','Premis Cd')
premise_code_dict = {k: v for k, v in premise_code_dict.items() if not (pd.isna(k) and pd.isna(v))}
status_code_dict = create_lookup_dict(df,'Status Desc','Status')
area_code_dict = create_lookup_dict(df,'AREA NAME','AREA')

In [111]:
df = remove_outliers(df)

In [80]:
df = impute_data(df)

In [88]:
#Putting crimes into broad categories added to a column names 'Crime Categories'
crime_mapping = {
       'VIOLATION OF COURT ORDER':'Violation of court order',
       'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)':'Vandalism',
       'OTHER MISCELLANEOUS CRIME':'Other', 'RAPE, ATTEMPTED':'Sexual Misconduct',
       'SHOPLIFTING - PETTY THEFT ($950 & UNDER)':'Shoplifting',
       'BURGLARY FROM VEHICLE':'Burglary',
       'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT':'Assault with Weapon',
       'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD':'Theft',
       'BATTERY - SIMPLE ASSAULT':'Assault', 'ROBBERY':'Robbery', 'BOMB SCARE':'Terrorism',
       'CHILD NEGLECT (SEE 300 W.I.C.)':'Crimes against Children',
       'INTIMATE PARTNER - AGGRAVATED ASSAULT':'Domestic Violence',
       'INTIMATE PARTNER - SIMPLE ASSAULT':'Domestic Violence',
       'THEFT PLAIN - PETTY ($950 & UNDER)':'Theft',
       'CRIMINAL THREATS - NO WEAPON DISPLAYED':'Threats', 'ATTEMPTED ROBBERY':'Robbery',
       'RAPE, FORCIBLE':'Sexual Misconduct', 'BRANDISH WEAPON':'Assault with Weapon',
       'THROWING OBJECT AT MOVING VEHICLE':'Vandalism',
       'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)':'Shoplifting',
       'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT':'Crimes against Children',
       'SHOTS FIRED AT INHABITED DWELLING':'Gun Crimes', 'VEHICLE - STOLEN':'Vehicle Theft',
       'KIDNAPPING - GRAND ATTEMPT':'Kidnapping', 'CRIMINAL HOMICIDE':'Homicide',
       'THEFT PLAIN - ATTEMPT':'Theft', 'THEFT, PERSON':'Theft', 'TRESPASSING':'Tresspassing',
       'KIDNAPPING':'Kidnapping', 'VEHICLE - ATTEMPT STOLEN':'Vehicle Theft',
       'BATTERY WITH SEXUAL CONTACT':'Sexual Misconduct', 'DOCUMENT FORGERY / STOLEN FELONY':'Forgery',
       'BATTERY POLICE (SIMPLE)':'Battery',
       'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT':'Crimes against Children',
       'DISTURBING THE PEACE':'Disorderly Conduct', 'OTHER ASSAULT':'Assault',
       'VIOLATION OF RESTRAINING ORDER':'Violation of Restraining Order', 'LEWD CONDUCT':'Sexual Misconduct',
       'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)':'Crimes against Children',
       'THEFT OF IDENTITY':'Identity Theft',
       'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER':'Assault with Weapon',
       'CHILD ANNOYING (17YRS & UNDER)':'Crimes against Children',
       'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH':'Sexual Misconduct',
       'ORAL COPULATION':'Sexual Misconduct', 'CRUELTY TO ANIMALS':'Crimes against Animals',
       'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)':'Theft',
       'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD':'Sexual Misconduct', 'PEEPING TOM':'Invasion of Privacy',
       'INDECENT EXPOSURE':'Sexual Misconduct', 'BIKE - STOLEN':'Vehicle Theft',
       'BURGLARY FROM VEHICLE, ATTEMPTED':'Burglary', 'BURGLARY, ATTEMPTED':'Burglary',
       'STALKING':'Stalking', 'THREATENING PHONE CALLS/LETTERS':'Threats',
       'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ':'Sexual Misconduct',
       'BUNCO, PETTY THEFT':'Theft', 'BUNCO, GRAND THEFT':'Theft',
       'VIOLATION OF TEMPORARY RESTRAINING ORDER':'Violation of Restraining Order',
       'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)':'Theft',
       'SEXUAL PENETRATION W/FOREIGN OBJECT':'Sexual Misconduct', 'EXTORTION':'Extortion', 'PICKPOCKET':'Pickpocketing',
       'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)':'Theft', 'PURSE SNATCHING':'Theft',
       'WEAPONS POSSESSION/BOMBING':'Terrorism', 'COUNTERFEIT':'Financial Fraud', 'FALSE IMPRISONMENT':'Crimes against Personal Liberty',
       'CREDIT CARDS, FRAUD USE ($950.01 & OVER)':'Financial Fraud',
       'DISCHARGE FIREARMS/SHOTS FIRED':'Gun Crimes', 'THEFT FROM PERSON - ATTEMPT':'Theft',
       'UNAUTHORIZED COMPUTER ACCESS':'Cyber Crime', 'PANDERING':'Sex Crimes', 'PIMPING':'Sex Crimes',
       'DRUNK ROLL - ATTEMPT':'Robbery', 'SHOPLIFTING - ATTEMPT':'Shoplifing',
       'RESISTING ARREST':'Resisting Arrest', 'DISHONEST EMPLOYEE - GRAND THEFT':'Theft',
       'RECKLESS DRIVING':'Traffic Offence', 'PROWLER':'Property Crime',
       'CREDIT CARDS, FRAUD USE ($950 & UNDER':'Financial Fraud',
       'DOCUMENT WORTHLESS ($200.01 & OVER)':'White-Collar Crimes', 'CHILD STEALING':'Crimes against Children',
       'CONTEMPT OF COURT':'Contemp of Court', 'CONSPIRACY':'Conspiracy',
       'THEFT FROM MOTOR VEHICLE - ATTEMPT':'Theft', 'DRUNK ROLL':'Robbery',
       'BATTERY ON A FIREFIGHTER':'Battery', 'ILLEGAL DUMPING':'Environmental Crime', 'LYNCHING':'Homicide',
       'THEFT, COIN MACHINE - PETTY ($950 & UNDER)':'Theft',
       'GRAND THEFT / INSURANCE FRAUD':'Theft', 'BUNCO, ATTEMPT':'Theft',
       'DISRUPT SCHOOL':'Disorderly Conduct', 'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)':'Theft',
       'FALSE POLICE REPORT':'Criminal Offence', 'PURSE SNATCHING - ATTEMPT':'Theft',
       'BOAT - STOLEN':'Theft',
       'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT':'Gun Crimes',
       'DRIVING WITHOUT OWNER CONSENT (DWOC)':'Traffic Offence', 'CONTRIBUTING':'Contributing',
       'FAILURE TO YIELD':'Traffic Offence', 'TILL TAP - ATTEMPT':'Theft', 'DRUGS, TO A MINOR':'Crimes against Children',
       'INCITING A RIOT':'Disorderly Conduct',
       'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM':'Sexual Misconduct',
       'BRIBERY':'Bribery', 'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)':'Theft',
       'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER':'Theft',
       'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01':'Theft',
       'TILL TAP - GRAND THEFT ($950.01 & OVER)':'Theft', 'LYNCHING - ATTEMPTED':'Homicide',
       'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE':'Sexual Misconduct', 'CHILD ABANDONMENT':'Crimes against Children',
       'LEWD/LASCIVIOUS ACTS WITH CHILD':'Crimes against Children',
       'DISHONEST EMPLOYEE - PETTY THEFT':'Theft', 'TELEPHONE PROPERTY - DAMAGE':'Vandalism',
       'PICKPOCKET, ATTEMPT':'Pickpocketing', 'TILL TAP - PETTY ($950 & UNDER)':'Theft',
       'DOCUMENT WORTHLESS ($200 & UNDER)':'White-Collar Crimes',
       'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS':'Sex Crimes',
       'MANSLAUGHTER, NEGLIGENT':'Homicide', 'GRAND THEFT / AUTO REPAIR':'Theft', 'BIGAMY':'Bigamy',
       'THEFT, COIN MACHINE - ATTEMPT':'Theft', 'PETTY THEFT - AUTO REPAIR':'Theft',
       'BIKE - ATTEMPTED STOLEN':'Theft', 'FAILURE TO DISPERSE':'Disorderly Conduct',
       'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)':'Gun Crimes',
       'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE':'Sex Crimes', 'CHILD PORNOGRAPHY':'Crimes against Children',
       'ABORTION/ILLEGAL':'Abortion', 'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)':'Sex Crimes',
       'DISHONEST EMPLOYEE ATTEMPTED THEFT':'Theft', 'TRAIN WRECKING':'Sabotage',
       'BLOCKING DOOR INDUCTION CENTER':'Disorderly Conduct',
       'FIREARMS RESTRAINING ORDER (FIREARMS RO)':'Gun Crimes',
       'VEHICLE - MOTORIZED SCOOTERS, BICYCLES, AND WHEELCHAIRS':'Vehicle Crimes',
       'FIREARMS TEMPORARY RESTRAINING ORDER (TEMP FIREARMS RO)':'Gun Crimes',
       'FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)':'Gun Crimes',
       'VANDALISM - MISDEAMEANOR ($399 OR UNDER)': 'Vandalism',
       'BURGLARY': 'Burglary',
        'ARSON': 'Arson'
}

descent_mapping = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}

df['Crime Categories'] = df['Crm Cd Desc'].map(crime_mapping)
df['Descent Categories'] = df['Vict Descent'].map(descent_mapping)

In [99]:
df = df.dropna()

In [122]:
df.to_csv('crime_data.csv',index=False)

In [129]:
df.shape

(2774864, 23)

In [2]:
df = pd.read_csv('crime_data.csv')

In [3]:
df.shape

(2774864, 23)

In [5]:
#changing column names

In [10]:
df = pd.read_csv('crime_data.csv')
new_column_names = {'DR_NO':'DR_No','Date Rptd':'Date Reported','DATE OCC':'Date Occurred','TIME OCC':'Time Occurred','AREA':'Area','AREA NAME':'Area Name','Rpt Dist No':'Reporting District No','Crm Cd':'Crime Code','Crm Cd Desc':'Crime Description','Vict Age':'Victim Age','Vict Sex':'Victim Sex','Vict Descent':'Victim Descent','Premis Cd':'Premise Code','Premis Desc':'Premise Description','Status Desc':'Status Description','Crm Cd 1':'Crime Code 1','LOCATION':'Location','LAT':'Latitude','LON':'Longitude'}
df = df.rename(columns=new_column_names)

In [11]:
df.to_csv('crime_data.csv')

#### Note: Crime Code is the same as Crime Code 1. One of them can be dropped.